In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

25/01/30 12:57:19 WARN Utils: Your hostname, Sagars-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.66 instead (on interface en0)
25/01/30 12:57:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/30 12:57:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

```
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
```

In [4]:
df_green.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-10 21:05:54|  2020-01-10 21:13:49|                 N|       1.0|         129|         129|            1.0|         0.81|        6.5|  0.5|    0.

In [5]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [6]:
rdd.take(5)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 10, 21, 5, 54), PULocationID=129, total_amount=8.51),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 17, 12, 33, 5), PULocationID=75, total_amount=18.36),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 30, 5, 41), PULocationID=117, total_amount=43.91),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 11, 14, 25, 44), PULocationID=41, total_amount=12.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 4, 14, 45, 19), PULocationID=129, total_amount=7.8)]

In [11]:
from datetime import datetime
start = datetime(year=2021, month=4, day=2)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [12]:
rdd.filter(filter_outliers).take(1)

[Row(lpep_pickup_datetime=datetime.datetime(2021, 10, 30, 4, 55, 9), PULocationID=33, total_amount=25.86)]

In [7]:
rdd.filter(lambda row: False).take(1)

[]

In [13]:
from datetime import datetime

In [14]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [15]:
rows = rdd.take(10)
row = rows[0]

In [16]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 10, 21, 5, 54), PULocationID=129, total_amount=8.51)

In [17]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [25]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .take(50)

[((datetime.datetime(2020, 1, 10, 21, 0), 129), (8.51, 1)),
 ((datetime.datetime(2020, 1, 17, 12, 0), 75), (18.36, 1)),
 ((datetime.datetime(2020, 1, 30, 5, 0), 117), (43.91, 1)),
 ((datetime.datetime(2020, 1, 11, 14, 0), 41), (12.3, 1)),
 ((datetime.datetime(2020, 1, 4, 14, 0), 129), (7.8, 1)),
 ((datetime.datetime(2020, 1, 15, 12, 0), 75), (8.76, 1)),
 ((datetime.datetime(2020, 1, 10, 17, 0), 66), (16.26, 1)),
 ((datetime.datetime(2020, 1, 16, 7, 0), 129), (6.8, 1)),
 ((datetime.datetime(2020, 1, 1, 12, 0), 41), (11.8, 1)),
 ((datetime.datetime(2020, 1, 20, 23, 0), 37), (34.04, 1)),
 ((datetime.datetime(2020, 1, 19, 5, 0), 129), (5.8, 1)),
 ((datetime.datetime(2020, 1, 12, 3, 0), 51), (30.7, 1)),
 ((datetime.datetime(2020, 1, 4, 6, 0), 255), (5.8, 1)),
 ((datetime.datetime(2020, 1, 27, 7, 0), 95), (8.15, 1)),
 ((datetime.datetime(2020, 1, 20, 1, 0), 86), (76.84, 1)),
 ((datetime.datetime(2020, 1, 7, 6, 0), 82), (18.94, 1)),
 ((datetime.datetime(2020, 1, 25, 3, 0), 182), (8.0, 1)),
 (

In [18]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [24]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .take(10)

[((datetime.datetime(2020, 1, 10, 21, 0), 129), (317.58000000000015, 19)),
 ((datetime.datetime(2020, 1, 4, 14, 0), 129), (471.95000000000016, 34)),
 ((datetime.datetime(2020, 1, 20, 23, 0), 37), (86.45, 3)),
 ((datetime.datetime(2020, 1, 19, 5, 0), 129), (132.72, 10)),
 ((datetime.datetime(2020, 1, 4, 6, 0), 255), (55.2, 4)),
 ((datetime.datetime(2020, 1, 13, 4, 0), 91), (169.14000000000001, 7)),
 ((datetime.datetime(2020, 1, 17, 16, 0), 25), (271.84000000000003, 19)),
 ((datetime.datetime(2020, 1, 13, 4, 0), 76), (164.59, 6)),
 ((datetime.datetime(2020, 1, 2, 1, 0), 85), (77.19999999999999, 3)),
 ((datetime.datetime(2020, 1, 3, 12, 0), 181), (171.49, 15))]

In [23]:
from collections import namedtuple

In [21]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [29]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [30]:
from pyspark.sql import types

In [33]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [37]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema)

In [39]:
df_result.write.parquet('data/tmp/green-revenue', mode='overwrite')

25/01/30 14:27:13 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/01/30 14:27:13 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/01/30 14:27:13 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/01/30 14:27:13 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/01/30 14:27:27 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/01/30 14:27:27 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/01/30 14:27:27 WARN MemoryManager: Total allocation exceeds 95.0

In [44]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [45]:
import pandas as pd

In [46]:
rows = duration_rdd.take(10)

In [47]:
df = pd.DataFrame(rows, columns=columns)

In [48]:
columns

['VendorID',
 'lpep_pickup_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_distance']

In [51]:
#model = ...

def model_predict(df):
    #y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [55]:
import pandas as pd
rows = duration_rdd.take(10)

In [57]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [59]:
duration_rdd.mapPartitions(apply_model_in_batch).toDF().take(10)

[Row(Index=0, VendorID=2, lpep_pickup_datetime=Row(), PULocationID=129, DOLocationID=129, trip_distance=0.81, predicted_duration=4.050000000000001),
 Row(Index=1, VendorID=2, lpep_pickup_datetime=Row(), PULocationID=75, DOLocationID=42, trip_distance=2.69, predicted_duration=13.45),
 Row(Index=2, VendorID=2, lpep_pickup_datetime=Row(), PULocationID=117, DOLocationID=188, trip_distance=13.11, predicted_duration=65.55),
 Row(Index=3, VendorID=2, lpep_pickup_datetime=Row(), PULocationID=41, DOLocationID=151, trip_distance=2.13, predicted_duration=10.649999999999999),
 Row(Index=4, VendorID=2, lpep_pickup_datetime=Row(), PULocationID=129, DOLocationID=260, trip_distance=0.89, predicted_duration=4.45),
 Row(Index=5, VendorID=2, lpep_pickup_datetime=Row(), PULocationID=75, DOLocationID=75, trip_distance=0.88, predicted_duration=4.4),
 Row(Index=6, VendorID=2, lpep_pickup_datetime=Row(), PULocationID=66, DOLocationID=232, trip_distance=2.25, predicted_duration=11.25),
 Row(Index=7, VendorID=2

In [60]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [61]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
| 4.050000000000001|
|             13.45|
|             65.55|
|10.649999999999999|
|              4.45|
|               4.4|
|             11.25|
|              4.55|
|             13.45|
|             26.75|
|               2.9|
|             39.15|
|               2.5|
|               4.5|
|119.39999999999999|
|               9.8|
|               0.0|
|7.1499999999999995|
|              4.25|
|             12.05|
+------------------+
only showing top 20 rows

